In [1]:
from clean_colored_lyrics import *
from synced_lyrics import *
from fuzzywuzzy import fuzz
from fuzzywuzzy import process
from fuzzysearch import find_near_matches
import regex
import datetime
import pandas as pd
import math
import os
from pathlib import Path
import copy
from datetime import datetime, timedelta, date
from difflib import SequenceMatcher

In [2]:
def get_lyrics(raw_html, debug):
    lyrics_container = raw_html.findAll('div', class_='Lyrics__Container-sc-68a46031-1 ibbPVY')

    found_section = False
    section_style = []
    assigned_lyrics = []
    members = []
    section = ''
    line = 0
    style_start = False

    for section in lyrics_container:
        if debug: print(section)
        for l in section:
            if l.name == 'div':
                continue
            
            if l.name == 'br':
                if debug: print('br')
                line += 1
                continue
            
            l_style = [l.name]
            l_text = l.get_text()
            if l_text.find('*') >= 0:
                l_style = ['*']
                if l_text.find('**') >= 0:
                    l_style = ['**']

            if l_text == '' or l_text.strip() == ',' or l_text.find('세븐틴') >= 0:
                continue
            
            if not l.name == None:
                for x in l.contents:
                    if not x.name == None and (x.name == 'i' or x.name == 'b'):
                        if debug: print("CHILDREN", x, x.name)
                        l_style.append(x.name)
                l_text = l.get_text(separator="\n", strip=True)
            
            if debug: print(l_style, l_text, found_section)

            if not found_section:
                if l_text.find('[') >= 0:
                    section_style = []
                    section = ''
                    line = 0
                    found_section = True

                    if l_text.find(']') >= 0:
                        found_section = False
                        l_text = l_text[:l_text.find(']')]
                    
                    section = l_text[1:l_text.find(':')]

                    member = l_text[l_text.find(':')+1:].replace(',','').strip()
                
                    if '8' in member:
                        i = member.find('8')
                        if member[i-1] == ' ':
                            member = member[:i-1] + member[i:]
                    
                    member = re.split(r'[&,\s]+', member)
                    if debug: print('new me', {'style': l_style, 'member': member})
                    section_style.append({'style': l_style, 'member': member})

                    for m in member:
                        if m not in members and not m == '':
                            members.append(m)

                else:
                    curr_member = next((style['member'] for style in section_style if set(style['style']) == set(l_style)), None)
                    
                    if l_style[0] == '*' or l_style[0] == '**':
                        l_text = l_text.replace(l_style[0], '')
                    l_text = l_text.replace('(', '').replace(')', '').strip()

                    while not l_text == '':
                        i = l_text.find('\n')
                        curr_t = l_text[:i]
                        l_text = l_text[i+1:]
                        if normalize_lyrics(curr_t).replace(',', '').strip() == '':
                            l_text == ''
                            break

                        if i == -1:
                            if debug: print('ADD LINE ---> ',{'member': curr_member, 'lyric': l_text, 'section': section, 'line': line})
                            assigned_lyrics.append({'member': curr_member, 'lyric': l_text, 'section': section, 'line': line})
                            break

                        if debug: print('ADD LINE ---> ', normalize_lyrics(curr_t), {'member': curr_member, 'lyric': curr_t, 'section': section, 'line': line})
                        assigned_lyrics.append({'member': curr_member, 'lyric': curr_t, 'section': section, 'line': line})
                        line += 1
            else:
                if l_text == ']':
                    found_section = False
                    continue

                if l_text.find(']') >= 0:
                    found_section = False
                    l_text = l_text[:l_text.find(']')]

                if debug: print('found', found_section, l_text)
                
                if l_text.find('*') >= 0:
                    pattern = r'(\*\*|\*)([^*]+)\1'
                    for marker, name in re.findall(pattern, l_text):
                        l_style = "**" if marker == "**" else "*"
                        pattern = fr'{re.escape(l_style)}(.*?){re.escape(l_style)}'
                        member = re.findall(pattern, l_text)[0]

                        if '8' in member:
                            i = member.find('8')
                            if member[i-1] == ' ':
                                member = member[:i-1] + member[i:]
                        member = re.split(r'[&,\s]+', member)
                        section_style.append({'style': [l_style], 'member': member})
                        if debug: print('new me', {'style': [l_style], 'member': member}, name)

                        for m in member:
                            if m not in members and not m == '':
                                members.append(m)
                else:
                    member = l_text[l_text.find(':')+1:].replace(',','').strip()

                    if '8' in member:
                        i = member.find('8')
                        if member[i-1] == ' ':
                            member = member[:i-1] + member[i:]
                    member = re.split(r'[&,\s]+', member)
                    if debug: print('new me', {'style': l_style, 'member': member})
                    section_style.append({'style': l_style, 'member': member})

                    for m in member:
                        if m not in members and not m == '':
                            members.append(m)

            if debug: print()

        if debug: print()

    return {'assigned_lyrics': assigned_lyrics, "members": members}

# results = get_lyrics(raw_html, True)

In [17]:
# with open('../json/albums_meta_data.json', 'r') as file:
#     data = json.load(file)

song_name = "Chilli"
artist = 'seventeen'
album = 'YOU MADE MY DAWN'

# song = next((s for s in data if s["name"] == song_name), None)
# print(song)
# album = song['album']

raw_html = get_soup("https://genius.com/Seventeen-"+ song_name.replace(" ", "-") + "-lyrics")
lyrics_results = get_lyrics(raw_html, False)


# raw_color_coded = get_raw_color_coded_html(song_name)
# colored_lyrics = get_lyrics(raw_color_coded)
# export_testing_html(str(raw_color_coded['main_lyrics_body']))
print(lyrics_results['members'])
meta_data = get_song_info(song_name, artist, album)
meta_data['unit'] = lyrics_results['members']

colored_lyrics = lyrics_results['assigned_lyrics']
synced_lyrics = meta_data['syncedLyrics']


['Vernon', 'S.COUPS', 'Mingyu', 'Wonwoo']
Chilli seventeen YOU MADE MY DAWN https://lrclib.net/api/search?artist_name=seventeen&track_name=Chilli&albumName=YOU_MADE_MY_DAWN
meta_data {'id': 23521230, 'name': 'Chilli', 'trackName': 'Chilli', 'artistName': 'SEVENTEEN', 'albumName': '24H', 'duration': 186.0, 'instrumental': False, 'plainLyrics': "Ooh, ah yeh yeh\n오늘 멋은 챙겨줘 yeah\n겉옷 챙겨서 나와 yeah\n생각보다 밖에 날씨 좋아 데리러 갈게 나와 yeah\n구석에 쌓아보자\n곧 더우니까\n무더운 여름마냥\n분위기 올려보자\n칠리 칠리해진 덕에 섭섭한 친구 불러 모아\n질리 질리도록 들은 음성 연결을 할 수 없어\n매년 매번 더 추워\n내년 겨울 앞에 붙여서\n올 한파보단 오늘이 더 더 덜 추워\n이불 덮어도 추워 추워 보일러를 틀어 틀어\n두 손 발은 얼어 꽁 oh eh oh\n친구들을 불러 불러 추워서 안 나와도\n나 혼자서 얼음 땡\n이런 날씨에\n아무도 안 나와 'cause outside's so chilly\nDoesn't stop us from breathin' 신바람 입김\nWatch me and my team\n사시사철 we just chillin'\n쏠리는 시선에 saucin' on 'em like chilli\nChilli (chilli)\nOh so chilli (chilli)\nOh so chilli\nYeah 쏠리는 시선 drip drip my chilli sauce\nDrip drip my chilli sauce\nI don't want no other sauce\n기 기분 so chilli 바람이 매섭네 oh\nSweet like chocol

In [4]:
print(meta_data)

{'id': 2921586, 'name': 'GoodtoMe', 'trackName': 'Good to Me', 'artistName': 'SEVENTEEN', 'albumName': 'Loved Tracks', 'duration': 190.0, 'instrumental': False, 'plainLyrics': "한 번쯤 그런 적 있잖아\n눈만 봐도 다 알 수가 있는 (whoa, yeah)\n뭘 어쩌나 싶다가도?\n이 감정을 알고 싶은걸 (oh)\n\n걸을 때 by my side (side)\n꽉 잡은 손도 놓치고 싶지가 않아\n설레는 기분은 다 너라서 매우 좋아 다\n내가 뭘 바라야 하니? 내 맘은 이미 충분히\n너로 너로 꽉 꽉 차 있네\n\nThis feels so right (so right)\n너만 있으면, 다 모든 게 좋아\n넌 나에게 쉴 새 없이 꿀 떨어지는 damage야\n철없이 아플 거야 난 (아야, 아야, 아야)\n\n'Cause you, you're my everything, 우린 알맞아\n'Cause you, you're my everyday, 너도 알잖아\n난 네가 필요해, 넌 내가 필요해\n'Cause you, you already know, everything is you, you\n\n(워어어, 워어어) good to me, good to me, 아야야\n(워어어, 워어어) good to me, good to me, 아야야\n(워어어, 워어어) good to me, good to me, 아야야\n(워어어, 워어어) 아야야, good to me\n\nYeah, 네가 내 맘에 sign하는 사이에\n나의 방화벽이 무너져 버린 지 오래\nPassword는 pass 넌 대체\n나에 대해 뭘 잘 알길래?\n내 속마음 모두 너에게 보이는지?\n\n걸을 때 by my side (side)\n꽉 잡은 손도 놓치고 싶지가 않아\n설레는 기분은 다 너라서 매우 좋아 다\n내가 뭘 바라야 하니? 내 맘은 이미 충분히\n너로 너로 꽉 꽉 차 있네\n\n

In [5]:
for c in colored_lyrics:
    print(c)
print()

for s in synced_lyrics:
    print(s)
print()
print(len(colored_lyrics), len(synced_lyrics))

{'member': ['DK'], 'lyric': '한 번쯤 그런 적 있잖아', 'section': 'Verse 1', 'line': 1}
{'member': ['DK'], 'lyric': '눈만 봐도 다 알 수가 있는', 'section': 'Verse 1', 'line': 2}
{'member': None, 'lyric': 'Woah, yeah', 'section': 'Verse 1', 'line': 2}
{'member': ['Seungkwan'], 'lyric': '뭘 어쩌나 싶다가도', 'section': 'Verse 1', 'line': 3}
{'member': ['Seungkwan'], 'lyric': '이 감정을 알고 싶은걸', 'section': 'Verse 1', 'line': 4}
{'member': None, 'lyric': 'Oh-oh', 'section': 'Verse 1', 'line': 4}
{'member': ['Dino'], 'lyric': '걸을 때 by my side', 'section': 'Refrain', 'line': 1}
{'member': None, 'lyric': 'Side', 'section': 'Refrain', 'line': 1}
{'member': ['Dino'], 'lyric': '꽉 잡은 손도 놓치고 싶지가 않아', 'section': 'Refrain', 'line': 2}
{'member': ['Dino'], 'lyric': '설레는 기분은 다 너라서 매우 좋아 다', 'section': 'Refrain', 'line': 3}
{'member': ['S.Coups'], 'lyric': '내가 뭘 바라야, 하니', 'section': 'Refrain', 'line': 4}
{'member': ['S.Coups'], 'lyric': '내 맘은 이미 충분히', 'section': 'Refrain', 'line': 5}
{'member': ['MINGYU'], 'lyric': '너로 너로 꽉꽉 차 있네', '

In [18]:
def merge_singular(colored_lyrics, synced_lyrics, looking_ahead_max, debug):    
    c_i = 0
    s_i = 0
    m_i = 0

    # colored_lyrics_copy = copy.deepcopy(colored_lyrics[:20])
    # synced_lyrics_copy = copy.deepcopy(synced_lyrics[:20])
    
    colored_lyrics_copy = copy.deepcopy(colored_lyrics)
    synced_lyrics_copy = copy.deepcopy(synced_lyrics)

    merged_lines = copy.deepcopy(colored_lyrics_copy)
    for m in merged_lines:
        m['lyric'] = m['lyric'].strip()
        m['start'] = ''
        m['end'] = ''
        m['delta'] = 0

    unmerged_line_color = []
    unmerged_line_synced = []

    c_looking_ahead_index = 0
    s_looking_ahead_index = 0

    c_len_shorter = True
    check_color = True

    while c_i < len(colored_lyrics) or s_i < len(synced_lyrics_copy):
        if debug: print('c_i',c_i, '  s_i', s_i, '  m_i', m_i, '  c_looking_ahead_index', c_looking_ahead_index, '    s_looking_ahead_index', s_looking_ahead_index)

        if c_i >= len(colored_lyrics) or c_looking_ahead_index > looking_ahead_max:
            if debug: print('MERGE REMAINING SYNCED')
            for s in synced_lyrics_copy[s_i:]:
                unmerged_line_synced.append(s)
                if debug: print('ADDING UMERGED SYNCED --->', s)
            break

        if s_i >= len(synced_lyrics_copy) or s_looking_ahead_index > looking_ahead_max:
            if debug: print('MERGE REMAINING COLOR')
            for c in colored_lyrics[c_i:]:
                unmerged_line_color.append(c)
                if debug: print('ADDING UMERGED COLOR --->', c)
            break


        if c_i + c_looking_ahead_index >= len(colored_lyrics):
            if debug: print("OVER COLOR")
            looking_ahead_max = max(0, len(merged_lines) - c_i - 1)
            if debug: print(looking_ahead_max, c_i, s_i)
            break
        elif s_i + s_looking_ahead_index >= len(synced_lyrics_copy):
            if debug: print("OVER SYNCED")
            looking_ahead_max = max(0, len(synced_lyrics_copy) - s_i - 1)
            if debug: print(looking_ahead_max, c_i, s_i)
            break

        c_line = colored_lyrics_copy[c_i + c_looking_ahead_index]
        s_line = synced_lyrics_copy[s_i + s_looking_ahead_index]
        c_lyric = c_line['lyric']
        c_lyric_norm = normalize_lyrics(c_lyric)
        s_lyric = s_line['lyric']
        s_lyric_norm = normalize_lyrics(s_lyric)
        m_line = merged_lines[m_i]
        if debug: print(c_line, s_line, m_line)
        if debug: print(c_lyric_norm, s_lyric_norm)

        max_l_dist = 5
        c_len = len(c_lyric_norm)
        s_len = len(s_lyric_norm)
        if c_len < 5 or s_len < 5:
            max_l_dist = 1
        elif c_len < 10 or s_len < 10:
            max_l_dist = int(min(c_len, s_len) / 3)

        find_match = []
        remaining_match = []
        remaining_str = ''
        if c_len <= s_len:
            c_len_shorter = True
            find_match = find_near_matches(c_lyric_norm, s_lyric_norm, max_l_dist=max_l_dist)
            remaining_str = s_lyric_norm
            if debug: print('c_len find_match: ', find_match)
        else:
            c_len_shorter = False
            find_match = find_near_matches(s_lyric_norm, c_lyric_norm, max_l_dist=max_l_dist)
            remaining_str = c_lyric_norm
            if debug: print('s_len find_match: ', find_match)

        if debug: print('found', find_match)
        found = False
        match_end = 0

        if find_match:
            found = find_match[0].start <= 8 and find_match[0].matched
            match_end = find_match[0].end
            find_match = found
        
        if not find_match:
            found = len(set(c_lyric_norm) ^ set(s_lyric_norm)) <= 1
            if debug and found: print('SAME SET OF LETTERS', found, set(c_lyric_norm), set(s_lyric_norm))
            if found: match_end = min(len(c_lyric), len(s_lyric))
            
            if not found:
                prefix = os.path.commonprefix([c_lyric, s_lyric])
                if debug: print('PREFIX: ' ,prefix)
                if len(prefix) >= 1: 
                    if debug: print('STARTS THE SAME')
                    match_end = len(prefix)
                    found = True
            

        remaining_str = remaining_str[match_end:].strip()
        print("REMINAING STR:" , remaining_str)

        if found:
            for i in range(s_looking_ahead_index):
                new_line = {
                    'member': ['adlib'],
                    'lyric': synced_lyrics_copy[s_i+i]['lyric'],
                    'start': synced_lyrics_copy[s_i+i]['start'],
                    'end': synced_lyrics_copy[s_i+i]['end'],
                    'delta': synced_lyrics_copy[s_i+i]['delta'],
                    'section': colored_lyrics_copy[max(0,c_i + c_looking_ahead_index - 1)]['section'],
                    'line': colored_lyrics_copy[max(0,c_i + c_looking_ahead_index - 1)]['line'],
                    # 'section_name': ''
                }
                merged_lines.insert(m_i + i, new_line)
                if debug: print("INSERT SYNCED MERGE --->", m_i + i, new_line )
                m_i += 1

            if c_looking_ahead_index and not m_line['lyric'] == c_line['lyric']:
                print('SKIPPING COLOR -->', m_i, m_line)
                m_i += c_looking_ahead_index

            m_line = merged_lines[m_i]
            print('NEW M:', m_i , m_line)

            if c_len_shorter and not remaining_str:
                if m_line['start'] == '':
                    m_line['start'] = s_line['start']
                m_line['end'] = s_line['end']
                m_line['delta'] = round(m_line['delta'] + s_line['delta'], 2)
                if debug: print('EXACT COLOR MERGE --->', m_i, m_line)
                c_i += (1 + c_looking_ahead_index)
                s_i += (1 + s_looking_ahead_index)
                m_i += 1
                s_looking_ahead_index = 0
                c_looking_ahead_index = 0

            if not c_len_shorter and not remaining_str:
                m_line = merged_lines[m_i]
                if m_line['start'] == '':
                    m_line['start'] = s_line['start']
                m_line['end'] = s_line['end']
                m_line['delta'] = round(m_line['delta'] + s_line['delta'], 2)

                if debug: print('EXACT SYNCED MERGE --->', m_i, m_line)
                c_i += 1
                s_i += (1 + s_looking_ahead_index)
                s_looking_ahead_index = 0
                c_looking_ahead_index = 0
                check_color = True
                m_i += 1

            if c_len_shorter and remaining_str:
                if m_line['start'] == '':
                    m_line['start'] = s_line['start']

                ratio = float(match_end) / float(len(s_lyric_norm))
                d = round(s_line['delta'] * ratio, 2)

                time_object = datetime.strptime(s_line['start'], '%M:%S:%f').time()
                new_start = str(((datetime.combine(datetime.now(), time_object)) + timedelta(seconds=d)).time())
                if debug: print('new_start', s_line['start'], (datetime.combine(datetime.now(), time_object)), new_start, d)

                if new_start.find('.') > 0:
                    pattern = r"\d{2}:(\d{2}:\d{2}:\d{2})"
                    new_start = re.findall(pattern, new_start.replace('.', ':'))[0]
                else:
                    pattern = r"(\d{2}:\d{2}:\d{2})"
                    new_start = re.findall(pattern, new_start)[0]
                if debug: print('new_start', s_line['start'], (datetime.combine(datetime.now(), time_object)), new_start, d)
                s_line['start'] = new_start
                s_line['delta'] = round(s_line['delta'] - d, 2)
                s_line['lyric'] = remaining_str

                m_line['delta'] = round(m_line['delta'] + d, 2)
                m_line['end'] = new_start
                check_color = True
                                
                c_i += (1 + c_looking_ahead_index)
                s_i += s_looking_ahead_index
                s_looking_ahead_index = 0
                c_looking_ahead_index = 0

                if debug: print('PARTIAL S_LINE MERGE --->', m_i, m_line)
                m_i += 1

            if not c_len_shorter and remaining_str:
                if m_line['start'] == '':
                    m_line['start'] = s_line['start']
                c_line['lyric'] = remaining_str
                m_line['delta'] = round(m_line['delta'] + s_line['delta'], 2)

                time_object = datetime.strptime(s_line['start'], '%M:%S:%f').time()
                new_end = str(((datetime.combine(datetime.now(), time_object)) + timedelta(seconds=m_line['delta'])).time())
                if new_end.find('.') > 0:
                    pattern = r"\d{2}:(\d{2}:\d{2}:\d{2})"
                    new_end = re.findall(pattern, new_end.replace('.', ':'))[0]
                else:
                    pattern = r"(\d{2}:\d{2}:\d{2})"
                    new_end = re.findall(pattern, new_end)[0]

                m_line['end'] = new_end

                s_i += (1 + s_looking_ahead_index)
                c_i += c_looking_ahead_index
                c_looking_ahead_index = 0
                s_looking_ahead_index = 0
                check_color = True

                if debug: print('PARTIAL C_LINE MERGE --->', m_i, m_line)
                # m_i += 1

        else:
            if check_color:
                c_looking_ahead_index += 1
                s_looking_ahead_index = 0
                if debug: print("CHECK COLOR")
            else: 
                s_looking_ahead_index += 1
                c_looking_ahead_index = 0
                if debug: print("CHECK SYNCED")

        if s_looking_ahead_index > looking_ahead_max:
            s_looking_ahead_index = 0
            if not check_color:
                # s_i += 1
                c_i += 1
                m_i += 1
                if debug: print("reset all: c = 0")
            else:
                new_line = {
                    'member': '',
                    'lyric': s_line['lyric'],
                    'start': s_line['start'],
                    'end': s_line['end'],
                    'delta': s_line['delta'],
                    'section': '',
                    'line': '',
                    # 'section_name': ''
                }
                merged_lines.insert(m_i, new_line)
                check_color = True
                if debug: print("reset all INSERT MERGE --->", m_i, new_line )
                m_i += 1

        if c_looking_ahead_index > looking_ahead_max:
            c_looking_ahead_index = 0
            if not check_color:
                c_i += 1
            else:
                check_color = False
            if debug: print("c_looking_ahead_index reset")
        
        if s_looking_ahead_index > looking_ahead_max:
            s_looking_ahead_index = 0
            s_i += 1
            if debug: print("s_looking_ahead_index reset")

        if debug: print()

    key_order = ['start', 'end', 'delta', 'section_name', 'section', 'line', 'member', 'lyric'] 
    merged_lines = [
        {k: entry[k] for k in key_order if k in entry} 
        for entry in merged_lines
    ]

    return merged_lines

meg = merge_singular(colored_lyrics, synced_lyrics, 2, 1)

c_i 0   s_i 0   m_i 0   c_looking_ahead_index 0     s_looking_ahead_index 0
{'member': ['Vernon'], 'lyric': 'Ooh', 'section': 'Intro', 'line': 1} {'start': '00:09:46', 'end': '00:14:46', 'lyric': 'Ooh, ah yeh yeh', 'delta': 5.0} {'member': ['Vernon'], 'lyric': 'Ooh', 'section': 'Intro', 'line': 1, 'start': '', 'end': '', 'delta': 0}
ooh oohahyehyeh
c_len find_match:  [Match(start=0, end=3, dist=0, matched='ooh')]
found [Match(start=0, end=3, dist=0, matched='ooh')]
REMINAING STR: ahyehyeh
NEW M: 0 {'member': ['Vernon'], 'lyric': 'Ooh', 'section': 'Intro', 'line': 1, 'start': '', 'end': '', 'delta': 0}
new_start 00:09:46 2026-02-03 00:00:09.460000 00:00:10.820000 1.36
new_start 00:09:46 2026-02-03 00:00:09.460000 00:10:82 1.36
PARTIAL S_LINE MERGE ---> 0 {'member': ['Vernon'], 'lyric': 'Ooh', 'section': 'Intro', 'line': 1, 'start': '00:09:46', 'end': '00:10:82', 'delta': 1.36}

c_i 1   s_i 0   m_i 1   c_looking_ahead_index 0     s_looking_ahead_index 0
{'member': ['Vernon'], 'lyric': 'A

In [19]:
for m in meg:
    print(m)

{'start': '00:09:46', 'end': '00:10:82', 'delta': 1.36, 'section': 'Intro', 'line': 1, 'member': ['Vernon'], 'lyric': 'Ooh'}
{'start': '00:10:82', 'end': '00:14:46', 'delta': 3.64, 'section': 'Intro', 'line': 2, 'member': ['Vernon'], 'lyric': 'Ah, ya-ya'}
{'start': '00:14:46', 'end': '00:16:00', 'delta': 1.54, 'section': 'Verse 1', 'line': 1, 'member': ['S.COUPS'], 'lyric': '오늘 멋은 챙겨줘, yeah'}
{'start': '00:16:00', 'end': '00:17:61', 'delta': 1.61, 'section': 'Verse 1', 'line': 2, 'member': ['S.COUPS'], 'lyric': '겉옷 챙겨서 나와, yeah'}
{'start': '00:17:61', 'end': '00:21:51', 'delta': 3.9, 'section': 'Verse 1', 'line': 3, 'member': ['S.COUPS'], 'lyric': '생각보다 밖에 날씨 좋아, 데리러 갈게, 나와, yeah'}
{'start': '00:21:51', 'end': '00:25:79', 'delta': 2.76, 'section': 'Verse 1', 'line': 4, 'member': ['S.COUPS'], 'lyric': '구석에 쌓아보자 Ayy, 곧 더우니까 Yeah'}
{'start': '00:24:27', 'end': '00:27:19', 'delta': 2.92, 'section': 'Verse 1', 'line': 5, 'member': ['S.COUPS'], 'lyric': '무더운 여름마냥 Yeah, 분위기 올려보자'}
{'start': '

In [20]:
def update_txt(album_name, track):
    with open('../json/raw_txt/'+ album_name.replace(' ', '_')+'_discography.txt', 'a') as file:
        file.write('\n')
        file.write('NEW UPDATE')
        file.write('\n')
        for t in track:
            json.dump(t, file, ensure_ascii=False)
            file.write('\n')
        file.write('\n')
        file.write('\n')
    print('wrote to: ../json/raw_txt/'+ album_name.replace(' ', '_')+'_discography.txt')

update_txt(album, meg)

wrote to: ../json/raw_txt/YOU_MADE_MY_DAWN_discography.txt


In [9]:
def look_ahead_color(c_line_og, s_line_og, debug):
    s_line = s_line_og.copy()
    s_lyric = normalize_lyrics(s_line['lyric']).strip()

    c_line = c_line_og.copy()
    c_line['delta'] = 'UNKNOWN'
    c_line['start'] = 'UNKNOWN'
    c_line['end'] = 'UNKNOWN'
    c_lyric = normalize_lyrics(c_line['lyric']).strip()


    if debug: print('s_line_before', s_line)
    if debug: print('c_line_before',c_line)

    found = False
    match_end = 0

    max_l_dist = 5
    if len(c_lyric) < 5 or len(s_lyric) < 5:
        max_l_dist = 1
    elif len(c_lyric) < 10 or len(s_lyric) < 10:
        max_l_dist = int(min(len(c_lyric), len(s_lyric)) / 3)
    
    if debug: print('max dist', max_l_dist, int(min(len(c_lyric), len(s_lyric))))
    find_match = find_near_matches(c_lyric, s_lyric, max_l_dist=max_l_dist)
    if debug: print('find_match: ', find_match)

    if find_match:
        found = find_match[0].start <= 3 and find_match[0].matched
        match_end = find_match[0].end
    else:
        prefix = os.path.commonprefix([c_lyric, s_lyric])
        if debug: print('PREFIX: ' ,prefix)
        if len(prefix) >= 4: 
            if debug: print('STARTS THE SAME')
            match_end = len(prefix)
            found = True

    if found:
        c_line['start'] = s_line['start']

        # If not exact match, update the start time for updated synced line
        if s_lyric[match_end:].strip():
            pattern = r"((\d{2}):(\d{2}):(\d{2}))"
            time = re.findall(pattern, s_line['delta'])[0]

            delta_as_int = int(time[1]) * 60 + int(time[2]) + int(time[3]) * 0.01
            ratio = float(match_end / float(len(s_lyric)))


            duration = delta_as_int * ratio
            duration = datetime.timedelta(seconds=duration)
            minute = int(duration.total_seconds()//60)
            sec = int(duration.total_seconds()%60)
            tenths = int(round((duration.total_seconds() % 1) * 100))
            c_line['delta'] = f"{minute:02}:{sec:02}:{tenths:02}"

            new_end_time = str_to_time(s_line['start'], '.') + duration
            new_end_time = f"{new_end_time.minute:02}:{new_end_time.second:02}:{new_end_time.microsecond//10000:02}"
            c_line['end'] = new_end_time
            
            duration = delta_as_int - delta_as_int * ratio
            duration = datetime.timedelta(seconds=duration)
            minute = int(duration.total_seconds()//60)
            sec = int(duration.total_seconds()%60)
            tenths = int(round((duration.total_seconds() % 1) * 100))
            s_line['delta'] = f"{minute:02}:{sec:02}:{tenths:02}"

        else:
            c_line['end'] = s_line['end']
            c_line['delta'] = getDelta(c_line, ':')
            
        c_line['lyric'] = c_line_og['lyric']

        s_line['lyric'] = s_lyric[match_end:].strip()
        s_line['start'] = c_line['end']

        if debug: print(c_line)
        if debug: print(s_line)

    return {'short_line': c_line, 'long_line': s_line, 'found': found}


In [10]:
def look_ahead_synced(s_line_og, c_line_og, debug):
    c_line = c_line_og.copy()
    c_lyric = normalize_lyrics(c_line['lyric'].lower()).strip()

    s_line = s_line_og.copy()
    s_line['member'] = 'UNKNOWN'
    s_line['section'] = 'UNKNOWN'
    s_line['line'] = 'UNKNOWN'
    s_lyric = normalize_lyrics(s_line['lyric'].lower()).strip()


    if debug: print('s_line_before', s_line)
    if debug: print('c_line_before',c_line)
    
    found = False
    match_end = 0

    max_l_dist = 5
    if len(c_lyric) < 5 or len(s_lyric) < 5:
        max_l_dist = 1
    elif len(c_lyric) < 10 or len(s_lyric) < 10:
        max_l_dist = int(min(len(c_lyric), len(s_lyric)) / 3)
    
    if debug: print('max dist', max_l_dist, int(min(len(c_lyric), len(s_lyric))))
    find_match = find_near_matches(s_lyric, c_lyric, max_l_dist=max_l_dist)
    if debug: print('find_match: ', find_match)

    if find_match:
        found = find_match[0].start <= 3 and find_match[0].matched
        match_end = find_match[0].end
    else:
        prefix = os.path.commonprefix([c_lyric, s_lyric])
        if debug: print('PREFIX: ' ,prefix)
        if len(prefix) >= 4: 
            if debug: print('STARTS THE SAME')
            match_end = len(prefix)
            found = True

    if found:
        pattern = r"((\d{2}):(\d{2}):(\d{2}))"
        time = re.findall(pattern, s_line['delta'])[0]

        delta_as_int = int(time[1]) * 60 + int(time[2]) + int(time[3]) * 0.01
        ratio = float(match_end) / float(len(c_lyric))

        duration = delta_as_int * ratio
        duration = datetime.timedelta(seconds=duration)
        minute = int(duration.total_seconds()//60)
        sec = int(duration.total_seconds()%60)
        tenths = int(round((duration.total_seconds() % 1) * 100))
        s_line['delta'] = f"{minute:02}:{sec:02}:{tenths:02}"

        new_end_time = str_to_time(s_line['start'], '.') + duration
        new_end_time = f"{new_end_time.minute:02}:{new_end_time.second:02}:{new_end_time.microsecond//10000:02}"
        # s_line['end'] = new_end_time
        s_line['member'] = c_line['member']
        s_line['section'] = c_line['section']
        s_line['line'] = c_line['line']
        
        c_line['lyric'] = c_lyric[match_end:].strip()
        if debug: print('s_line', s_line)
        if debug: print('c_line',c_line)

        
    return {'short_line': s_line, 'long_line': c_line, 'found': found}


In [11]:
def merge_singular_old(colored_lyrics, synced_lyrics, looking_ahead_max, debug):    
    merged_lines = []

    unmerged_line_color = []
    unmerged_line_synced = []

    c_i = 0
    s_i = 0

    # colored_lyrics_copy = colored_lyrics[78:100].copy()
    # synced_lyrics_copy = synced_lyrics[50:70].copy()

    colored_lyrics_copy = colored_lyrics.copy()
    synced_lyrics_copy = synced_lyrics.copy()

    looking_ahead_index = 0
    looking_ahead = []

    find_children = False
    parent = True

    default_merge_choice_is_color = False

    if len(colored_lyrics) > len(synced_lyrics):
        default_merge_choice_is_color = True
    else:
        default_merge_choice_is_color = False

    if debug: print(default_merge_choice_is_color, len(colored_lyrics_copy), len(synced_lyrics_copy))

    while c_i < len(colored_lyrics_copy) or s_i < len(synced_lyrics_copy):
        
        if debug: print('c_i',c_i, '  s_i', s_i, '  looking_ahead_index', looking_ahead_index)

        if c_i >= len(colored_lyrics_copy):
            if debug: print('MERGE REMAINING SYNCED')
            for s in synced_lyrics_copy[s_i:]:
                unmerged_line_synced.append(s)
                if debug: print('ADDING UMERGED SYNCED --->', s)
            break

        if s_i >= len(synced_lyrics_copy):
            if debug: print('MERGE REMAINING COLOR')
            for c in colored_lyrics_copy[c_i:]:
                unmerged_line_synced.append(c)
                if debug: print('ADDING UMERGED COLOR --->', c)
            break
        
        if c_i + looking_ahead_max >= len(colored_lyrics_copy):
            if debug: print("OVER")
            looking_ahead_max = max(0, len(colored_lyrics_copy) - c_i - 1)
            if debug: print(looking_ahead_max, c_i, s_i)
            # break
        elif s_i + looking_ahead_max >= len(synced_lyrics_copy):
            if debug: print("OVER")
            looking_ahead_max = max(0, len(synced_lyrics_copy) - s_i - 1)
            if debug: print(looking_ahead_max, c_i, s_i)
            # break
        
        c_line = colored_lyrics_copy[c_i + looking_ahead_index]
        s_line = synced_lyrics_copy[s_i]

        c_lyric = c_line['lyric']
        c_lyric_norm = normalize_lyrics(c_lyric)
        s_lyric = s_line['lyric']
        s_lyric_norm = normalize_lyrics(s_lyric)
        if debug: print(c_lyric_norm, s_lyric_norm)

        if c_lyric_norm == s_lyric_norm:
            if debug: print(c_lyric, s_lyric)
            new_line = {'start': s_line['start'], 'end': s_line['end'], 'lyric': c_lyric, 'member': c_line['member'], 'delta': s_line['delta'], 'section': c_line['section'], 'line': c_line['line']}
            merged_lines.append(new_line)
            
            if debug: print("EXACT MATCH: ", new_line)
            if debug: print()
            c_i += (looking_ahead_index + 1)
            s_i += 1
            looking_ahead_index = 0
        else:
            color_is_short = True
            if debug: print("LENGETH: ",len(c_lyric_norm),len(s_lyric_norm))
            if len(c_lyric_norm) <= len(s_lyric_norm):
                if debug: print('color shorter')
                looking_ahead = look_ahead_color(c_line, s_line, debug)
            else:
                color_is_short = False
                if debug: print('synced shorter')
                looking_ahead = look_ahead_synced(s_line, c_line, debug)

            if looking_ahead_index:
                parent = color_is_short

            if debug: print('LOOK AROUND --->', looking_ahead)
            if looking_ahead['found']:
                merged_lines.append(looking_ahead['short_line'])
                if debug: print('MERGE --->', looking_ahead['short_line'])

                if (looking_ahead_index <= looking_ahead_max) :
                    if debug: print("inc colored inside")
                    for l in range(looking_ahead_index):
                        unmerged_line_color.append(colored_lyrics_copy[c_i+l])
                        if debug: print('ADDING UMERGED COLOR: ', l, ' --->', colored_lyrics_copy[c_i+l])
                            
                if color_is_short:
                    if not looking_ahead['long_line']['lyric']:
                        s_i += 1
                        c_i += (looking_ahead_index + 1)
                        if debug: print("EXACT MATCH")
                        find_children = False
                                
                    else:
                        old = synced_lyrics_copy[s_i]
                        synced_lyrics_copy[s_i] = looking_ahead['long_line']
                        if debug: print("PARTIAL MATCH ---> replacing in synced ", old, synced_lyrics_copy[s_i], ' at ', s_i)

                        c_i += (1 + looking_ahead_index)
                        find_children = True
                else:
                    if not looking_ahead['long_line']['lyric']:
                        c_i += (1 + looking_ahead_index)
                        s_i += 1
                        if debug: print("EXACT MATCH")
                        find_children = False
                    else:
                        old = colored_lyrics_copy[c_i + looking_ahead_index]
                        colored_lyrics_copy[c_i + looking_ahead_index] = looking_ahead['long_line']
                        if debug: print("PARTIAL MATCH ---> replacing in colored ", old, colored_lyrics_copy[c_i + looking_ahead_index], ' at ', c_i + looking_ahead_index)
                        s_i += 1
                        c_i += looking_ahead_index
                        find_children = True

                looking_ahead_index = 0
            else:
                if debug: print('not found', looking_ahead_index)
                looking_ahead_index += 1

        if looking_ahead_index > looking_ahead_max:
            if find_children:
                if parent:
                    if debug: print("inc colored")
                    c_i += 1
                    unmerged_line_color.append(colored_lyrics_copy[c_i])
                    if debug: print('ADDING UMERGED COLOR --->', colored_lyrics_copy[c_i])
                else:
                    if debug: print("inc short")
                    unmerged_line_synced.append(synced_lyrics_copy[s_i])
                    if debug: print('ADDING UMERGED SYNCED --->', synced_lyrics_copy[s_i])
                    s_i += 1
            else:
                if debug: print('ADDING UMERGED COLOR --->', colored_lyrics_copy[c_i])
                if debug: print('ADDING UMERGED SYNCED --->', synced_lyrics_copy[s_i])

                unmerged_line_color.append(colored_lyrics_copy[c_i])
                unmerged_line_synced.append(synced_lyrics_copy[s_i])
                c_i += 1
                s_i += 1
            find_children = False
            if debug: print('rest looking_ahead_index')
            
            looking_ahead_index = 0
        
        if debug: print()

    key_order = ['start', 'end', 'delta', 'section', 'line', 'member', 'lyric']

    merged_lines = [
        {k: entry[k] for k in key_order if k in entry} 
        for entry in merged_lines
    ]
    unmerged_line_color = [
        {k: entry[k] for k in key_order if k in entry} 
        for entry in unmerged_line_color
    ]
    unmerged_line_synced = [
        {k: entry[k] for k in key_order if k in entry} 
        for entry in unmerged_line_synced
    ]

        
    return {"merged_lines": merged_lines, "unmerged_line_color": unmerged_line_color, "unmerged_line_synced": unmerged_line_synced}

# merge_singular(colored_lyrics, synced_lyrics, 1, True)

In [12]:
def main():
    with open('../json/urls.json', 'r') as file:
        data = json.load(file)

    print(data)

    overall_data = []
    debug = False

    for d in data:
        song_name = d['name']
        artist = 'seventeen'
        album = d['album']

        print(song_name)
        raw_color_coded = get_raw_color_coded_html(song_name)
        colored_lyrics = get_colored_lyrics(raw_color_coded)
        export_testing_html(str(raw_color_coded['main_lyrics_body']))
        meta_data = get_song_info(song_name, artist, album)

        if meta_data == 'song not found':
            print('song not found')
            continue
        
        meta_data['color_key'] = raw_color_coded['color_key']
            
        meta_data['unit'] = raw_color_coded['unit']
        synced_lyrics = meta_data['syncedLyrics']

        looking_ahead_max = 1
        merged = merge_singular(colored_lyrics, synced_lyrics, looking_ahead_max, debug)
        meta_data['syncedLyrics'] = merged['merged_lines']
        meta_data['unmerged_line_color'] = merged['unmerged_line_color']
        meta_data['unmerged_line_synced'] = merged['unmerged_line_synced']
        
        song_name = song_name.replace(' ', '_')
        with open('../json/songs/'+ meta_data['name'].lower() +'.json', 'w') as file:
                json.dump(meta_data, file, indent=4)

        print()
    
    print('done')

    #     overall_data.append(meta_data)

    # with open('../json/meta_data.json', 'w') as file:
    #         json.dump(overall_data, file, indent=4)
# main()

In [13]:
def singular():        
        with open('../json/urls.json', 'r') as file:
                data = json.load(file)

        overall_data = []
        debug = False

        song_name = input()
        artist = 'seventeen'

        song = next((s for s in data if s["name"] == song_name), None)
        print(song)
        album = song['album']

        raw_color_coded = get_raw_color_coded_html(song_name)
        colored_lyrics = get_colored_lyrics(raw_color_coded)
        export_testing_html(str(raw_color_coded['main_lyrics_body']))
        meta_data = get_song_info(song_name, artist, album)

        meta_data['color_key'] = raw_color_coded['color_key']
        
        meta_data['unit'] = raw_color_coded['unit']
        synced_lyrics = meta_data['syncedLyrics']

        looking_ahead_max = 1
        merged = merge_singular(colored_lyrics, synced_lyrics, looking_ahead_max, debug)
        meta_data['syncedLyrics'] = merged['merged_lines']
        meta_data['unmerged_line_color'] = merged['unmerged_line_color']
        meta_data['unmerged_line_synced'] = merged['unmerged_line_synced']

        song_name = song_name.replace(' ', '_')
        with open('../json/songs/'+ meta_data['name'].lower() +'.json', 'w') as file:
                json.dump(meta_data, file, indent=4)
# singular()

In [14]:
# print(raw_color_coded)

In [15]:
directory_path = Path('../json/albums')
files = [p for p in directory_path.iterdir() if p.is_file()]

meta_data = []
for file in files:
    print(file)

    with open(file, 'r') as f:
        data = json.load(f)

    meta_data.append(data)

# with open('../json/songs/lalali.json', 'r') as f:
#     meta_data = json.load(f)

with open('../json/albums_meta_data.json', 'w') as file:
        json.dump(meta_data, file, indent=4)

../json/albums/BOYS_BE_meta_data.json
../json/albums/17_CARAT_meta_data.json
../json/albums/YOU_MAKE_MY_DAY_meta_data.json
../json/albums/Going_Seventeen_meta_data.json
../json/albums/Heng:garæ_meta_data.json
../json/albums/FML_meta_data.json
../json/albums/SPILL_THE_FEELS_meta_data.json
../json/albums/Al1_meta_data.json
